In [1]:
#pip install datasets

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("zwhe99/DeepMath-103K")

/Users/macpro/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/macpro/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_ds = ds['train']
train_ds[1]

{'question': 'Find the auxiliary equation for the ordinary differential equation with constant coefficients: \\((x^2D^2 + xD + 1)y = \\sin(2\\log x)\\sin(\\log x)\\).',
 'final_answer': 'm^2 + 1 = 0',
 'difficulty': 5.0,
 'topic': 'Mathematics -> Differential Equations -> Ordinary Differential Equations (ODEs)',
 'r1_solution_1': 'Okay, so I need to find the auxiliary equation for the given ordinary differential equation (ODE) with constant coefficients: \\((x^2D^2 + xD + 1)y = \\sin(2\\log x)\\sin(\\log x)\\). Hmm, let me start by recalling what an auxiliary equation is. From my classes, I remember that for linear ODEs with constant coefficients, we often use characteristic equations, which are also called auxiliary equations. These are polynomial equations obtained by replacing the differential operator \\(D\\) with a variable, usually \\(m\\), and solving for \\(m\\). The roots of this equation help determine the general solution to the homogeneous equation.\n\nBut wait, the given d

# Preliminary tasks

## Briefly describe the data

In [4]:
# TODO

## Perform cluster analysis on questions field

In [5]:
# TODO

## Perform document index on different fields

In [ ]:
# pip install -q python-terrier==0.11.0 #now there is another version but we don't use it

: 

In [ ]:
column_names = train_ds.column_names
print(column_names)

# 2. Initialize PyTerrier (only once)
import pyterrier as pt

# google colab
#if not pt.started():
#  pt.init()

# vs code in mac
if not pt.java.started():
    pt.init()

['question', 'final_answer', 'difficulty', 'topic', 'r1_solution_1', 'r1_solution_2', 'r1_solution_3']


### Finding "quadratic form" in 'question'

In [ ]:
# 3. Build a corpus iterator over the 'question' field
def get_question_corpus(ds):
    for i, example in enumerate(ds):
        yield {
            'docno': str(i),                          # unique document ID
            'question': example['question']           # only the question text
        }

# 4. Index into a new folder, specifying that we only want the 'question' field
pt_index_path = './terrier_deepmath_questions'
indexer = pt.index.IterDictIndexer(pt_index_path, overwrite=True, meta_reverse=[])
index_ref = indexer.index(
    get_question_corpus(train_ds),
    fields=('question',),
    meta=('docno', 'question')  # store docno + question in the metadata
)

# 5. Load the index and run a TF-IDF retrieval over the 'question' field
index = pt.IndexFactory.of(index_ref)
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
results = tfidf.search("quadratic form")
print(results.head(6))

### Perform a 'question' search in only documents with x difficulty

In [ ]:
# Imposto difficulty = 0.0
filtered_ds = [ex for ex in train_ds if float(ex['difficulty']) == 0.0]

# Build a corpus iterator over the 'question' field
def get_question_corpus(ds):
    for i, example in enumerate(ds):
        yield {
            'docno': str(i),                          # unique document ID
            'question': example['question']           # only the question text
        }

# Index into a new folder, specifying that we only want the 'question' field
pt_index_path = './terrier_deepmath_questions'
indexer = pt.index.IterDictIndexer(pt_index_path, overwrite=True, meta_reverse=[])
index_ref = indexer.index(
    get_question_corpus(filtered_ds),
    fields=('question',),
    meta=('docno', 'question')
)

# Load the index and run a TF-IDF retrieval over the 'question' field
index = pt.IndexFactory.of(index_ref)
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
query = "quadratic form"
query1 = "quadratic form optimization minimize" #puoi mettere piu parole e funziona
results = tfidf.search(query)
print(results.head(6))

### Finding 'quadratic form' in more fields (NON RIUSCITO)

## Generate embeddings and analyze them

In [ ]:
# TODO

# Training

## M1C: Base model (topic classification)

## M1R: Base model (difficulty regression)

## M1S: Base model (short answer)

## M2: Introduce reasoning

## Comparisons